In [1]:
import pandas as pd

In [2]:
import sqlite3

In [3]:
conn = sqlite3.connect("sales.db")

In [ ]:
sql_query = """
SELECT 
    c.CustomerID AS Customer,
    c.Age,
    i.ItemName AS Item,
    SUM(s.Quantity) AS Quantity
FROM Sales s
JOIN Customers c ON s.CustomerID = c.CustomerID
JOIN Items i ON s.ItemID = i.ItemID
WHERE c.Age BETWEEN 18 AND 35
  AND s.Quantity IS NOT NULL
GROUP BY c.CustomerID, c.Age, i.ItemName
HAVING SUM(s.Quantity) > 0
ORDER BY c.CustomerID;
"""
df_sql = pd.read_sql_query(sql_query, conn)   #running query and loading dataframe

df_sql.to_csv("output.csv", sep = ";", index=False)    #to csv

In [ ]:
# Load tables into Pandas
customers = pd.read_sql_query("SELECT * FROM Customers;", conn)
items = pd.read_sql_query("SELECT * FROM Items;", conn)
sales = pd.read_sql_query("SELECT * FROM Sales;", conn)

# Merge tables
merged = sales.merge(customers, on="CustomerID").merge(items, on="ItemID")

# Filter by age 18–35
filtered = merged[(merged["Age"] >= 18) & (merged["Age"] <= 35)]

# Replace NULL quantities with 0
filtered["Quantity"] = filtered["Quantity"].fillna(0)

# Group and sum
df_pandas = (
    filtered.groupby(["CustomerID", "Age", "ItemName"], as_index=False)["Quantity"]
    .sum()
)

# Exclude rows with 0 quantity
df_pandas = df_pandas[df_pandas["Quantity"] > 0]

# Rename columns for consistency
df_pandas.rename(columns={
    "CustomerID": "Customer",
    "ItemName": "Item"
}, inplace=True)

# Save to CSV
df_pandas.to_csv("output_pandas.csv", sep=";", index=False)